## Домашнее задание 9
### Градиентный бустинг

В данном домашнем задании мы изучим применение градиентного бустинга для решения реальных задач.
  
Мы будем использовать данные об аренде квартир сервиса Яндекс.Недвижимость: по информации из объявления предсказывать время его экспозиции на сервисе.  

Метрикой качества для в данном задании является средняя абсолютная ошибка, MAE.

In [0]:
! wget https://www.dropbox.com/s/psutl0zafq50828/data.tsv > ./data.tsv

In [0]:
! pip install catboost==0.22
! pip install lightgbm==2.3.1

In [3]:
import pandas as pd
import catboost
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('./data.tsv', sep='\t')
num_features = ['agent_fee', 'floor', 'floors_total', 'kitchen_area', 'living_area', 'price', 'rooms_offered', 'total_area', 'total_images']
cat_features = ['balcony', 'building_type', 'month', 'renovation', 'studio']

X_train, X_test, y_train, y_test = train_test_split(data[num_features+cat_features], data['exposition_time'], test_size = 0.3, shuffle = False)

__Задание 1.__

Обучите реализации градиентного бустинга LightGBM и Catboost на **вещественных** признаках без подбора параметров. Сделайте предсказания для тестовой выборки, посчитайте MAE. Выведите модуль разности между значениями метрик LightGBM и Catboost. Ответ округлите до тысячных.

В этом и последующих экспериментах советуем измерять время обучения моделей.

In [1]:
%%time


Wall time: 0 ns


In [23]:
X_train[num_features].head()

,agent_fee,floor,floors_total,kitchen_area,living_area,price,rooms_offered,total_area,total_images
0,60.0,7,9.0,9.0,31.0,30000,2,48.0,12
1,50.0,5,17.0,-1.0,-1.0,55000,2,60.0,12
2,50.0,8,17.0,12.0,-1.0,45000,2,68.0,13
3,60.0,7,16.0,10.0,20.0,25000,1,40.0,12
4,50.0,7,14.0,8.0,30.0,50000,2,40.0,16


In [22]:
X_train[cat_features].head()

,balcony,building_type,month,renovation,studio
0,0,4,June,0,False
1,0,4,June,0,False
2,0,4,June,0,False
3,0,6,June,0,False
4,0,1,June,0,False


In [10]:
%%time

lightCLF = lightgbm.LGBMRegressor()
lightCLF.fit(X_train[num_features], y_train)
lightMAE = mean_absolute_error(y_test, lightCLF.predict(X_test[num_features]))
lightMAE

Wall time: 1.03 s


36.498598720526246

In [11]:
%%time
catCLF = catboost.CatBoostRegressor(logging_level='Silent')
catCLF.fit(X_train[num_features], y_train)
catMAE = mean_absolute_error(y_test, catCLF.predict(X_test[num_features]))
catMAE

Wall time: 22.2 s


36.35187053009413

In [12]:
round(abs(lightMAE-catMAE), 3)

0.147

__Задание 2.__

Подберите оптимальные параметры для Catboost на вещественных признаках,  используя все доступные комбинации из:

* глубины деревьев {5, 7, 9};
* темпа обучения {0.05, 0.1, 0.5}.

В качестве оптимизируемого функционала в алгоритме используйте MAE (loss_function='MAE'), random_seed=0, другие параметры оставьте без изменений.

Сделайте предсказания для тестовой выборки лучшей моделью, вычислите MAE.

In [27]:
import numpy as np

In [15]:
%%time

depth_params = [5, 7, 9]
rates_params = [0.05, 0.1, 0.5]

results = []

for depth in depth_params:
    for rate in rates_params:
        estimator = catboost.CatBoostRegressor(max_depth =  depth, learning_rate = rate, random_seed = 0,  logging_level = 'Silent', loss_function = 'MAE')
        estimator.fit(X_train[num_features], y_train)
        mae = mean_absolute_error(y_test, estimator.predict(X_test[num_features]))
        results.append([depth, rate, mae])


Wall time: 5min 30s


In [16]:
for result in results:
    print(result)

[5, 0.05, 31.641329495403955]
[5, 0.1, 31.666597555668933]
[5, 0.5, 32.03449635593659]
[7, 0.05, 31.65088635072659]
[7, 0.1, 31.718876336853853]
[7, 0.5, 32.20464448102391]
[9, 0.05, 31.699908737773335]
[9, 0.1, 31.81082673689335]
[9, 0.5, 32.980140337677646]


__Задание 3.__

Пока что мы использовали только вещественные данные, однако категориальные переменные также могут быть полезны для решения задачи. Перебирая комбинации параметров из предыдущего пункта, используйте все доступные признаки, обрабатывая категориальные встроенным методом Catboost.

Выведите значение MAE на тестовой выборке для лучшей модели.

In [32]:
%%time
results = []

for depth in depth_params:
    for rate in rates_params:
        estimator = catboost.CatBoostRegressor(max_depth=depth, learning_rate=rate, random_seed=0,  logging_level='Silent', loss_function='MAE')
        estimator.fit(X_train, y_train, cat_features=cat_features)
        mae = mean_absolute_error(y_test, estimator.predict(X_test))
        results.append([depth, rate, mae])

Wall time: 18min 46s


In [33]:
for result in results:
    print(result)

[5, 0.05, 32.3722251326191]
[5, 0.1, 34.930778136450144]
[5, 0.5, 36.91337869277467]
[7, 0.05, 32.81788765740183]
[7, 0.1, 34.48198449635924]
[7, 0.5, 42.54822213650881]
[9, 0.05, 34.5786073876299]
[9, 0.1, 36.20209330451235]
[9, 0.5, 44.83007795462268]


__Задание 4.__

Реализуйте блендинг (получение ответов нескольких моделей и взятие их с весами (их нужно подбирать на обучающей выборке)) полученных в заданиях 2 и 3 моделей и выведите MAE на тестовой выборке.

Обратите внимание: в этом задании нет ограничений на веса у моделей.

In [34]:
from sklearn.linear_model import LinearRegression

In [35]:
#LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)def weights(y_true, Y1, Y2):
    metric = []
    grid = np.linspace(0, 1, 1000)
    for weight0 in grid:
      weight1 = 1 - weight0
      y = Y1 * weight0 + Y2 * weight1

      metric.append([rmse(y_true, y), weight0, weight1])

    return metric

__Задание 5.__

В задании 3 вы подобрали гиперпараметры для CatBoost на всех признаках. Визуализируйте их важности в виде горизонтального bar-plot (отсортируйте признаки по убыванию важности, подпишите названия признаков по оси y).

Для каждого из двух алгоритмов удалите неважные признаки (значения менее 0.05; обычно по bar-plot хорошо видно порог, с которого начинается хвост неважных признаков) и обучите модель с теми же параметрами на оставшихся признаках. Выведите разность между значениями MAE на тестовой выборке до и после удаления признаков.

In [11]:
estimator = catboost.CatBoostRegressor(max_depth = 5, learning_rate = 0.05, random_seed = 0,  logging_level = 'Silent', loss_function = 'MAE')
estimator.fit(X_train, y_train, cat_features = cat_features)
print(f"mae: {mean_absolute_error(y_test, estimator.predict(X_test))}")

mae: 32.3722251326191


In [13]:
print(num_features+cat_features)
estimator.feature_importances_

['agent_fee', 'floor', 'floors_total', 'kitchen_area', 'living_area', 'price', 'rooms_offered', 'total_area', 'total_images', 'balcony', 'building_type', 'month', 'renovation', 'studio']


array([17.34436089,  0.49708245,  0.74474754, 11.31970037,  7.03450376,
       12.03881981,  0.47959563,  3.87594468,  5.58370211,  1.20354311,
        1.74517022, 20.27755389, 17.85527554,  0.        ])

In [14]:
estimator = catboost.CatBoostRegressor(max_depth = 5, learning_rate = 0.05, random_seed = 0,  
                                       logging_level = 'Silent', loss_function = 'MAE')
estimator.fit(X_train.drop(['studio'], axis = 1), y_train, cat_features = ['balcony', 'building_type', 'month', 'renovation'])
abs(mean_absolute_error(y_test, estimator.predict(X_test.drop(['studio'], axis = 1))) -32.3722251326191)

0.5067856709724126